<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 100; padding-top: 22px; text-align:right;">Jokin Cuesta Arrillaga</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# IA: DETECTOR DE CANSANCIO CON MEDIAPIPE Y OPENCV

Autor: Jokin Cuesta Arrillaga.

**Importante:** este proyecto se ha realizado en base al vídeo: *AI Body Language Decoder with MediaPipe and Python in 90 Minutes* de Nicholas Renotte. El proyecto se ha llevado a cabo con el único fin de desarrollar habilidades y en **ningún caso ha sido para obtener ningún beneficio económico.**

>A lo largo de este proyecto se utilizará  Mediapipe para estimar puntos de referencia tanto faciales como corporales, obtenidos directamente de una WebCam a través de la OpenCV . Con esos datos, se crearán modelos personalizados de clasificación de poses que le permitan decodificar lo que una persona podría estar diciendo con su lenguaje corporal con gran precisión, adaptando a las necesidades de cada aplicación. En este proyecto en concreto, se tratará de realizar detección de somnolencia para conductores.

---


**Antes de nada, instalaremos todas las dependencias que necesitaremos en este proyecto. Esta es la lista con las librerías necesarias:**

 - **MediaPipe:** ofrece soluciones de ML como el reconocimiento facial, mallas faciales, holístico, poses, detección de objetos, Motion Tracking etc. Para nuestra propuesta, utilizaremos el holístico, lo que nos permitirá reconocer en vivo de la pose humana simultánea , los puntos de referencia faciales y el seguimiento de mano. Obtendremos puntos coordenadas numéricas de articulaciones los cuales guardaremos en un .csv para su posterior tratamiento. Se puede apreciar mejor en este .gif obtenido de https://google.github.io/mediapipe/solutions/holistic.html.

<div style="float: centre; width: 100%;">
<img src="https://google.github.io/mediapipe/images/mobile/holistic_sports_and_gestures_example.gif" align="centre">
</div> 

 - **OpenCV:** Lo utilizaremos para obtener, renderizar y procesar la imagen de la cámara.
 - **Pandas:** Para el tratamiento de los datos obtenidos.
 - **Scikit-learn:** Para llevar adelante todos los modelos de ML.

In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

>Nota: a día 20/6/2022, existe un problema con las versiones de protobuf cuando se quiere cargar mediapipe. Para solucionarlo, se debe cargar la versión anterior con *!pip install --upgrade protobuf==3.20.1*.

In [ ]:
#!pip install --upgrade protobuf==3.20.1
import mediapipe as mp    #Importamos mediapipe
import cv2                #Importamos opencv

In [ ]:
mp_drawing = mp.solutions.drawing_utils   #Ayudas para dibujar
mp_holistic = mp.solutions.holistic       #Soluciones de Mediapipe

<a id="ej1"></a>

## 1. REALIZAR ALGUNAS DETECCIONES

En este apartado se realizarán algunas detecciones para la comprobación del correcto funcionamiento.

In [ ]:
# Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()      
        
        # Reformatear: BGR -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el modelo holistico:
        results = holistic.process(image)               
        
        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Mostramos la imagen                
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Se observa en la imagen siguiente que la detección es correcta.
    
---

<div style="float: centre; width: 100%;">
<img src="https://i.ibb.co/bRwV3C3/holistic.png" align="centre">
</div>

    
---
Ahora, todas las landmarks se guardan en results. Si se escribe directamente el valor de results, obtendremos '*mediapipe.python.solution_base.SolutionOutputs*'. Pero, results está construido por 4 modelos: face_landmarks, right_hand_landmarks, left_hand_landmarks y pose landmarks. 

    
    
---
    
Todos los landmarks del pose que detecta MediaPipe (33) se recogen en la siguiente imagen:

![]()

<div style="float: centre; width: 100%;">
<img src="https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png" align="centre">
</div>


---
<div class="alert alert-block alert-info">
Ahora, si compilamos *results.face_landmarks.landmark[0]*, obtendremos las coordenadas en 3D de la nariz:
</div>

In [ ]:
results.face_landmarks.landmark[0]

<div class="alert alert-block alert-info">
Y podemos obtener los valores x,y,z:
</div>

In [ ]:
print("x:", results.pose_landmarks.landmark[0].x,"y:", 
      results.pose_landmarks.landmark[0].y,"z:", 
      results.pose_landmarks.landmark[0].z)

<div class="alert alert-block alert-info">
Ahora pasaremos al siguiente apartado para almacenar los datos obtenidos de results en un .csv

</div>

<a id="ej2"></a>

    
---

## 2. CAPTURAR LANDMARKS Y EXPORTAR A ARCHIVO CSV



Importaremos 3 librerías para realizar el almacenamiento de datos.
 - **csv:** nos permitirá trabajar con archivos .csv y escribir nuestros datos en ellos.
 - **os:** usaremos para guardar los archivos en una carpeta específica. 
 - **numpy:** utilizaremos numpy arrays para procesar la información.

In [ ]:
#Importamos librerías necesarias:
import csv
import os
import numpy as np

<div class="alert alert-block alert-info">
Queremos cuatro valores de los landmarks: x,y,z,v; siendo v la visibilidad.  El modelo facial no dispone del valor de visibilidad, por lo que para todos los puntos de la cara tendremos un v con valor 0.
</div> 

---
 
<div class="alert alert-block alert-info">
Vamos a loopear por todos los landmarks, por lo que nos interesa saber cuántos son. Ya hemos visto que para el pose teníamos 33 landmarks. Ahora nos faltará los de la cara.
</div>

In [ ]:
#Tenemos 501 coordenadas en total
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

<div class="alert alert-block alert-info">
Clasificaremos los coordenadas de cada landmark con su número de identificación.
</div>

In [ ]:
#la primera columna servirá de identificación de clase 
landmarks = ['class']

for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

#Mostramos ejemplo:
print(landmarks[0:13])

<div class="alert alert-block alert-info">
Ahora empezaremos con la exportación al archivo .csv. Escribiremos los nombres de las columnas.
</div>

In [ ]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

<div class="alert alert-block alert-info">
Ahora, obtenemos un .csv (en la misma carpeta de este archivo) en el cual tenemos las columnas nombradas. 
</div>

---

<div class="alert alert-block alert-info">
Vamos ahora a recopilar datos. Para ello, necesitamos dos clases, el de cansado y despierto. Para cada clase, recogeremos los datos desde la cámara
</div>

In [ ]:
class_name = "Cansado"

In [ ]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<a id="ej3"></a>

## 3. ENTRENAR MODELO UTILIZANDO Scikit Learn

### 3.1 CARGAR DATOS CON PANDAS

Ahora necesitamos otras dos librerías para entrenar el modelo.
 - Pandas: para el procesamiento del dataframe (df)
 - Sklearn: en concreto *train_test_split*, con el cual dividiremos el dataset en dos: uno para el entrenamiento y el segundo para validar los modelos. 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Leemos el archivo .csv
df = pd.read_csv("coords.csv")
df

In [ ]:
# Guardaremos las variables en X y el target en Y
X = df.drop('class',axis=1)
y = df['class']

In [ ]:
# Dividimos el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1000)

### 3.2 MODELO DE CLASIFICACIÓN

Para crear el modelo, necesitaremos importar varias librerías. Cabe mencionar que, para que no dependamos únicamente de un modelo de ML, se va a realizar 4 pipelines de modelos. 
 - **sklearn.pipeline:** nos permitirá llevar a cabo esos 4 pipelines.
 - **sklearn.preprocessing:** Para la estandarización.
 - **sklearn.linear_model:** Para importar los modelos lineales *LogisticRegression* y *RidgeClassifier*.
 - **sklearn.ensemble:** Para importar modelos de ensemble como *RandomForestClassifier* y *GradientBoostingClassifier*.

In [ ]:
#Importamos las funciones necesarias
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

Realizaremos 4 modelos de ML para no depender únicamente de una.
 - Regresión Logística
 - Ridge Classifier
 - Random Forest Classifier
 - Gradient Boosting Classifier

In [ ]:
# Definimos los cuatro pipelines
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
# Guardamos los modelos en la biblioteca fit_models
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
# Probamos a predecir X_test con el modelo de Random Forest
fit_models['rf'].predict(X_test)

### 3.3 EVALUACIÓN Y SERIALIZACIÓN DEL MODELO

Ahora es hora de evaluar los modelos construidos. Para ello importaremos:
- **sklearn.metrics:** la función *accuracy_score* para obtener la precisión obtenida de cada modelo al predecir los *targets* de X_test.
- **pickle:** para la serialización del modelo.


In [ ]:
from sklearn.metrics import accuracy_score
import pickle

<div class="alert alert-block alert-info">
A continuación se predicen las clases de X_test para cada modelo.
</div>

In [ ]:
#Predecir las clases de X_test
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    
    #Mostramos las precisiones de cada modelo
    print(algo, accuracy_score(y_test, yhat))

<div class="alert alert-block alert-info">
Se ve que para los primeros dos (*LogisticRegression*, *RidgeClassifier*), la precisión ha sido del 100%. Para *GradientBoostingClassifier* y *RandomForestClassifier* 99.6%. Por elección propia, utilizaré el *RandomForestClassifier*.
</div>

---
    
<div class="alert alert-block alert-info">
Se serializa entonces el modelo obtenido en un archivo .pkl:
</div>

In [ ]:
# Escribimos el modelo en un archivo .pkl
with open('cansado_despierto.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4.  DETECCIONES Y ALERTA AL USUARIO
<div class="alert alert-block alert-info">
Vamos a ir a la parte final de este proyecto. Realizaremos las detecciones de cansado o despierto con el modelo entrenado. Para ello, volvemos a abrir el modelo serializado.
</div>

In [ ]:
# Abrimos de nuevo el modelo
with open('cansado_despierto.pkl', 'rb') as f:
    model = pickle.load(f)

<div class="alert alert-block alert-info">
De nuevo se abrirá la cámara y con la ayuda de MediaPipe haremos detecciones de los landmarks. El modelo será capaz de predecir el estado del/la usuario/a con esos datos. En la pantalla, vamos a mostrar la clase predicha y su probabilidad.

---

Cuando la probabilidad de estar cansado supere el 70%, vamos a alertar al usuario que necesita un descanso en un rectángulo rojo, como símbolo de alerta.
</div>

In [ ]:
#Conectamos la cámara
cap = cv2.VideoCapture(0)

# Inicia el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        
        #Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # BGR -> RGB y seguridad de la imagen.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Detecciones
        results = holistic.process(image)
        
        # RGB -> BGR y seguridad de la imagen.        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Exporta coordenadas
        try:
            # Extraemos Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
            
            # Extraemos Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
            
            # Concatenamos las filas
            row = pose_row+face_row

            # Hacemos detecciones
            X = pd.DataFrame([row])
            
            # Predecir la clase
            body_language_class = model.predict(X)[0]     
            
            # Predecir la probabilidad de la clase
            body_language_prob = model.predict_proba(X)[0]
            
            # Recogemos coordenadas de la oreja izquierda para visualizar texto dinámicamente
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            
            #Mostramos la clase en la oreja izq.
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Status Box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Mostramos Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Mostramos Probabilidad
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Al estar muy cansado, activamos la alerta de descanso.
            if (round(body_language_prob[np.argmax(body_language_prob)],2) > 0.70) 
            and (body_language_class.split(' ')[0] == "Cansado"):
                
                cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len('MUY CANSADO')*20, coords[1]-30), 
                          (0, 0, 255), -1)
                
                cv2.putText(image, 'MUY CANSADO', coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.rectangle(image, (0,0), (550, 60), (0, 0, 255), -1)
                
                cv2.putText(image, "ALERTA! NECESITAS DESCANSAR", (15,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)               

            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 5. CONCLUSIONES y PUNTOS A MEJORAR

En este proyecto hemos realizado un detector de cansancio, como aplicación de seguridad para los/as conductores en la carretera. Los resultados obtenidos han sido satisfactorios (se muestran en el siguiente .gif) y podrían mejorarse teniendo más datos entrenados o tomando en cuenta más variables en cuanto al cansancio. Incluso se podría estudiar más clases donde añadiríamos distintos grados de cansancio.

<div style="float: centre; width: 100%;">
<img src="https://github.com/Jokin-Cuesta-Arrillaga/Detector-de-Cansancio-con-MediaPipe-y-OpenCV/blob/main/Imagenes/Cansadodespierto.gif?raw=true" align="centre">
</div>